In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn 
from keras.layers import Dense
from keras.models import Sequential
import os
import matplotlib.pyplot as plt
import math

In [2]:
df = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/1000 points per trajectory/May10rd1percent_RLV_data 1000 points.csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
input = output =df.values
X = input[:,0:7]
y = output[:,7:9]
# Individual Data
from sklearn import preprocessing
X_norm = preprocessing.minmax_scale(X)
y_norm = preprocessing.minmax_scale(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.1, random_state=42)
X_val = X_train[-10000:]
y_val = y_train[-10000:]
X_train = X_train[:-10000]
y_train = y_train[:-10000]

In [3]:
R0 = 6378*1000;
g0=9.81
Cd = 0.5
S_ref =10.75
Isp =300;
I_z = 3346393
l_com = 9.778
t_ref = 32
h0=5000;
v0=325;
s0=850;
## The inputs and outputs which varied have values in decimals 
## rest all have been rounded off to 5-6 digits
omega0= 0.342
theta0= round(-75*math.pi/180,4)
gamma0= round(-75*math.pi/180,4) #
m0= round(26229.667,0) # our values of mass are rounded of when taking data
r0 = h0+6378000

## Here we are finding the trajectory which relates to the given input conditions

In [4]:
# First we find the index of the row where we have the given initial conditions, we validate if we had 
# taken the correct trajectory for that we need to check the ending values of s such that s = s0.
idx = df.loc[(df['h'] == h0) & (df['v'] == v0)]
idx = np.array(idx.index.astype(int))
def trajectory_finding(idx,df,s0):
    # we have 3003 points for getting full trajectory starting from index 0-3002
    for i in range(idx.shape[0]):
        df2= df[idx[0]+3003*i:idx[0]+3003*(i+1)]
        if (df2.s.values[3002]==s0):
            break
        else:
            continue
    return np.array([df2.index[0],df2.index[3002]])

In [5]:
idx_i,idx_f = trajectory_finding(idx,df,s0)

In [6]:
## Normalizing the inputs
def norm(a,b):
    return (a - min(b))/(max(b) - min(b))
def norm_inputs(a,b):
    return (a-min(b))/(max(b)-min(b))

In [7]:
h = X[:,0];
v = X[:,1];
s = X[:,2];
omega = X[:,3]
gamma= X[:,4]
m = X[:,5]
theta = X[:,6]
T = y[:,0]
b = y[:,1]
r = 6378000+h

h_norm = norm_inputs(h,h)
v_norm= norm_inputs(v,v)
s_norm= norm_inputs(s,s)
omega_norm= norm_inputs(omega,omega)
gamma_norm= norm_inputs(gamma,gamma)
m_norm= norm_inputs(m,m)
theta_norm= norm_inputs(theta,theta)
r_norm = norm_inputs(r,r)
T_norm = norm_inputs(T,T)
b_norm = norm_inputs(b,b)

h1 = norm_inputs(h0,h)
v1= norm_inputs(v0,v)
s1= norm_inputs(s0,s)
omega1= norm_inputs(omega0,omega)
gamma1= norm_inputs(gamma0,gamma)
m1= norm_inputs(m0,m)
theta1= norm_inputs(theta0,theta)
r1 = norm_inputs(r0,r)

**For this trajectory we are creating a vector of Neural Network inputs and obtain the Thrust, beta predicted values.
    given all the 7 states as input. and these controls will be sent to the DNN driven trajectory generation model**
    
    ip_traj = df.all
    normalize the inputs and ouputs for observation

In [8]:
data_norm = np.array([h_norm,v_norm,s_norm,omega_norm,gamma_norm,m_norm,theta_norm,T_norm,b_norm]).T
df_norm = pd.DataFrame(data_norm,columns=['h','v','s','omega','gamma','m','theta','Thrust','beta'])
# Trajectory for the given initial condition:
data_traj = df_norm.values[idx_i:idx_f+1][:,0:9]
ip_traj = data_traj[:,0:7]
op_traj = data_traj[:,7:9]

# Intermediate trajectories averaged initial guess 

In [9]:
## Intermediate trajectory values:
ip_mid = [];
op_mid = [];
for i in range(len(ip_traj)-1):
     ip_mid = np.append(ip_mid,(ip_traj[i,:]+ip_traj[i+1,:])/2)
     op_mid = np.append(op_mid,(op_traj[i,:]+op_traj[i+1,:])/2)
        
ip_mid = ip_mid.reshape(len(ip_traj)-1,7)
op_mid = op_mid.reshape(len(ip_traj)-1,2)

## Now we need to append the values for all state variables in DNN driven trajectory and then we need to antinormalize the outputs to validate with the actual trajectory.

In [10]:
def AntiNorm(Norm_value,actual):
  return (max(actual)-min(actual))*Norm_value+min(actual)

## Defining variables which will be appended as the iterations progress in RK4 method:

In [11]:
# local variables within the DNN function
ω2 = [];
θ2 = [];
v2 = [];
γ2 = [];
r2 = [];
s2 = [];
h2 = [];

In [21]:
# global variables for appending
ω_dnn = [];
θ_dnn = [];
v_dnn = [];
γ_dnn = [];
r_dnn = [];
s_dnn = [];
h_dnn = [];
m_dnn = [];
ω_int_dnn = [];
θ_int_dnn = [];
v_int_dnn = [];
γ_int_dnn = [];
r_int_dnn = [];
s_int_dnn = [];
h_int_dnn = [];
m_int_dnn = [];
b_int_dnn = [];
T_int_dnn = [];

# Giving the initial trajectory conditions 

In [22]:
h1 = h_norm[idx_i]
s1 = s_norm[idx_i]
v1 = v_norm[idx_i]
omega1 = omega_norm[idx_i]
m1 = m_norm[idx_i]
theta1 = theta_norm[idx_i]
gamma1 = gamma_norm[idx_i]
r1 = r_norm[idx_i] # this is < 1 since 1 is 5025.2

In [23]:
ip = np.array([h1,v1,s1,omega1,gamma1,m1,theta1]).reshape(1,7)
# T1,b1 = np.squeeze(model.predict(ip))
# T1_int,b1_int = np.squeeze(model.predict(ip_int.reshape(1,7)))

ip_int = ip_mid[0,:]
op_int = op_mid[0,:]

T1,b1 = np.squeeze(model.predict(ip))
T1_int,b1_int = np.squeeze(model.predict(ip_int.reshape(1,7)))
print(T1,b1, T1_int,b1_int)

## Substituting the actual values and validating if we get the same DNN

In [24]:
T1,b1 = op_traj[0,:] # 1st row of outputs for the given inputs
T1_int,b1_int = op_mid[0,:] # 1st row of intermediate values
print(T1,b1, T1_int,b1_int)

0.29138459162398 0.6510435332560345 0.2897757986048547 0.6501396648044693


# Start here

## Generating the trajectory from outputs using RK4

In [25]:
T1,b1 = op_traj[0,:] # 1st row of outputs for the given inputs
T1_int,b1_int = op_mid[0,:] # 1st row of intermediate values
b1_an = AntiNorm(b1,b)
b1_int_an = AntiNorm(b1_int,b)

T2,b2 = op_traj[1,:]
b2_an = AntiNorm(b2,b)
n= 3003
h_step = 1/n # since 1 trajectory has 3003 points.

print(b1,
theta1,
gamma1,
omega1)
## Before we start using them in RK4 Method we should antinormalize the angles
b1_an = AntiNorm(b1,b) 
theta1_an = AntiNorm(theta1,theta)
gamma1_an = AntiNorm(gamma1,gamma)
omega1_an = AntiNorm(omega1,omega)
print(b1_an,theta1_an, gamma1_an, omega1_an)

0.6510435332560345 0.627155523320357 0.5447154471544716 0.9578695516488089
0.058088 -1.309 -1.309 0.34199999999999997


#### These values should be initialized as zero at the start, they will be automatically updated 

In [26]:
df.head(1)

,h,v,s,omega,gamma,m,theta,Thrust,beta,time
0,4974.8,323.37,2.430700e-16,0.342,-1.309,26230,-1.309,479450.0,0.049258,0.0


In [27]:
##############################################################################
si = s_norm[idx_i]
gamma1_int = ip_mid[0,4]
theta1_int = ip_mid[0,6]
# taken values from intermediate stuff to send as inputs, but these should be same as initial values. As time progresses 
# they get updated by iterations.
g1_int_an = AntiNorm(gamma1_int,gamma) # gamma _intermediate antinormalized
t1_int_an = AntiNorm(theta1_int,theta) # theta _intermediate antinormalized
r1_int = r1 + h_step/2 * v1* np.sin(g1_int_an) * (t_ref *v0/R0)
##############################################################################

In [28]:
# def RK4():
for _ in range(n):
    ω_dnn = np.append(ω_dnn,omega1_an)
    n = np.append(ω_dnn,omega1_an)
    c2 = (t_ref*m0*g0*l_com/I_z)
    kω1 = - T1*np.sin(b1_an)*c2 # these values obtained are w.r.t tprime so to get w.r.t we need to divide by 32
    kω2 = - T1_int*np.sin(b1_int_an)*c2
    kω3 = - T1_int*np.sin(b1_int_an)*c2
    kω4 = - T2*np.sin(b2_an)*c2
    omega2_an = omega1_an + h_step/6 * (kω1 + 2*kω2 + 2*kω3 + kω4) 
    omega_int = (omega1_an + omega2_an)/2
    ω_int_dnn = np.append(ω_int_dnn,omega_int)
    print("predicted",omega2_an,"actual", AntiNorm(ip_traj[1,3],omega))

    b_int_dnn = np.append(b_int_dnn,b1_int_an)
    T_int_dnn = np.append(T_int_dnn,T1_int)

    theta1_an = AntiNorm(theta1,theta)
    θ_dnn = np.append(θ_dnn,theta1_an)
    kθ1 = t_ref*omega1_an
    kθ2 = t_ref*omega_int # intermediate trajectories
    kθ3 = t_ref*omega_int
    kθ4 = t_ref*omega2_an
    theta2_an =  theta1_an + h_step/6* (kθ1 +2*kθ2 + 2*kθ3 + kθ4)
    theta_int = (theta1_an + theta2_an)/2
    θ_int_dnn = np.append(θ_int_dnn,theta_int)
    print("predicted",theta2_an,"actual", AntiNorm(ip_traj[1,-1],theta))

    m_dnn = np.append(m_dnn,m1)
    km1 = -(t_ref/Isp)*T1
    km2 = -(t_ref/Isp)*T1_int
    km3 = -(t_ref/Isp)*T1_int
    km4 = -(t_ref/Isp)*T2
    m2 = m1 + h_step/6* (km1 + 2*km2 + 2*km3 + km4)
    m1_int = (m2 + m1)/2
    m_int_dnn = np.append(m_int_dnn,m1_int)
    print("predicted", m2 ,"actual ", m_norm[idx_i+1])

    v_dnn = np.append(v_dnn,v1)

    c1 = 0.5*1.225*Cd*(v0)**2 *S_ref / (m0*g0)
    D1 = c1 * v1**2
    # Intermediate of r1 is obtained from Euler method

    kv1 = -(g0*t_ref/v0) *(((T1 * np.cos(b1_an - gamma1_an + theta1_an) + D1)/m1) + (np.sin(gamma1_an)/(r1**2)))

    D1_int = c1 * (v1 + h_step*kv1/2)**2
    kv2 = -(g0*t_ref/v0) *(((T1_int * np.cos(b1_int_an - g1_int_an + t1_int_an) + D1_int/m1_int) + (np.sin(g1_int_an)/(r1_int**2))))

    D2_int = c1 * (v1 + h_step*kv2/2)**2
    kv3 = -(g0*t_ref/v0) *(((T1_int * np.cos(b1_int_an - g1_int_an + t1_int_an) + D2_int/m1_int) + (np.sin(g1_int_an)/(r1_int**2))))

    D3_int = c1 * (v1+h_step*kv3)**2
    gamma2_an = gamma1_an + -(g0*t_ref/v0) *(((T1 * np.sin(b1_an - gamma1_an + theta1_an))/(m1*v1)) + (np.cos(gamma1_an)/(r1**2 * v1)))
    kv4 = -(g0*t_ref/v0) *(((T2 * np.cos(b2_an - gamma2_an + theta2_an) + D3_int)/m2) + (np.sin(gamma2_an)/(r1**2)))

    v2 = v1 + h_step/6 * ( kv1 + 2*kv2 + 2*kv3 + kv4)

    v1_int = (v1 + v2)/2
    v_int_dnn = np.append(v_int_dnn,v1_int)
    print("predicted", v2 ,"actual ", v_norm[idx_i+1])

    γ_dnn = np.append(γ_dnn,gamma1_an)

    g1 = gamma1_an
    kγ1 = -(g0*t_ref/v0) *(((T1 * np.sin(b1_an - g1 + theta1_an))/(m1*v1)) + (np.cos(g1)/(r1**2 * v1)))

    g2 = g1_int_an + (h_step * kγ1 / 2)
    kγ2 = -(g0*t_ref/v0) *((T1_int * np.sin(b1_int_an - g2 + t1_int_an)/(m1_int*v1_int)) + (np.cos(g2)/(r1_int**2*v1_int)))

    g3 = g1_int_an + (h_step * kγ2 / 2)
    kγ3 = -(g0*t_ref/v0) *((T1_int * np.sin(b1_int_an - g3 + t1_int_an)/(m1_int*v1_int)) + (np.cos(g3)/(r1_int**2*v1_int)))

    g4 = g1_int_an + (h_step * kγ3)
    kγ4 = -(g0*t_ref/v0) *(((T2 * np.sin(b2_an - g4 + theta2_an) + D3_int)/(m2*v2)) + (np.cos(g4)/(r1**2*v2)))

    gamma2_an = gamma1_an + h_step/6 * ( kv1 + 2*kv2 + 2*kv3 + kv4)
    g1_int_an = (gamma2_an + gamma1_an)/2
    γ_int_dnn = np.append(γ_int_dnn,g1_int_an) 
    print("predicted", gamma2_an ,"actual ", AntiNorm(gamma_norm[idx_i+1],gamma))

    s_dnn = np.append(s_dnn,si)
    c4 = (v0*t_ref/s0)
    ks1 = c4 * v1 * np.cos(gamma1_an)
    ks2 = c4 * v1_int * np.cos(g1_int_an)
    ks3 = ks2
    ks4 = c4 * v2 * np.cos(gamma2_an)
    s2 = si + (h_step/6) * (ks1 + 2*ks2 + 2*ks3 + ks4)
    s1_int = (s1+s2)/2
    s_int_dnn = np.append(s_int_dnn,s1_int)
    print("predicted", s2 ,"actual ", s_norm[idx_i+1])

    r_dnn = np.append(r_dnn,r1)
    c3 = (v0*t_ref/R0)
    kr1 = c3 * v1 * np.sin(gamma1_an)
    kr2 = c3 * v1_int * np.sin(g1_int_an)
    kr3 = kr2
    kr4 = c3 * v2 * np.sin(gamma2_an)

    r2 = r1 + (h_step/6) * (kr1 + 2*kr2 + 2*kr3 + kr4)
    r1_int = (r1 + r2)/2
    r_int_dnn = np.append(r_int_dnn,r1_int)
    print("predicted", r2 , "actual ", r_norm[idx_i+1])

    h_dnn = np.append(h_dnn,h1)
    h2 = norm(AntiNorm(r2,r)-6378000,h)
    h1_int = (h1+h2)/2
    h_int_dnn = np.append(h_int_dnn,h1_int)

    r1 = r2;
    si = s2;
    gamma1_an = gamma2_an;
    h1 = h2;
    m1 = m2;
    theta1_an = theta2_an;
    omega1_an = omega2_an;
    v1= v2;
    idx_i = idx_i+1
    # ip = np.array([h1,v1,s1,omega1,gamma1,m1,theta1]).reshape(1,7)


predicted 0.34186600922280036 actual 0.34014
predicted -1.3053563582592191 actual -1.3012
predicted 0.9999897071533407 actual  0.9991434689507495
predicted 0.9947253362446254 actual  0.9933463980228231
predicted -1.3092067332774313 actual  -1.3093
predicted 0.001048657592230473 actual  0.0022261624830689905
predicted 0.9949847577975265 actual  0.9935525023302021
predicted 0.34173201844560075 actual 0.34014
predicted -1.3053577860663688 actual -1.3012
predicted 0.9999794143066814 actual  0.9982869379014989
predicted 0.994518777900776 actual  0.9917918181216135
predicted -1.3094132916212806 actual  -1.3095
predicted 0.0020962821076467627 actual  0.004446764827096206
predicted 0.9949842360843995 actual  0.9921197250702265
predicted 0.34159802766840114 actual 0.34014
predicted -1.3053592138735184 actual -1.3012
predicted 0.999969121460022 actual  0.9974304068522484
predicted 0.9943124096741724 actual  0.990237238220404
predicted -1.3096196598478842 actual  -1.3098
predicted 0.0031428810632

predicted 0.3387842213472093 actual 0.34014
predicted -1.305389197823661 actual -1.3012
predicted 0.9997529716801763 actual  0.9811563169164882
predicted 0.9900222053591443 actual  0.9589279990100436
predicted -1.3139098641629123 actual  -1.3178
predicted 0.024886724724367704 actual  0.051755823039953665
predicted 0.9949727784368568 actual  0.9612354152450313
predicted 0.3386502305700097 actual 0.34014
predicted -1.3053906256308105 actual -1.3012
predicted 0.999742678833517 actual  0.9802997858672376
predicted 0.9898199643503869 actual  0.9575288770989551
predicted -1.3141121051716698 actual  -1.3183
predicted 0.025911070434290966 actual  0.05383156324690699
predicted 0.9949722585420533 actual  0.959862337037516
predicted 0.3385162397928101 actual 0.34014
predicted -1.3053920534379602 actual -1.3012
predicted 0.9997323859868577 actual  0.9796573875802997
predicted 0.9896179084551414 actual  0.9560986635898421
predicted -1.3143141610669153 actual  -1.3188
predicted 0.026934418515722574 

predicted 0.33570243347161827 actual 0.34014
predicted -1.305422037388103 actual -1.3012
predicted 0.9995162362070119 actual  0.9635974304068522
predicted 0.9854171201022265 actual  0.9278053093878282
predicted -1.3185149494198296 actual  -1.3314
predicted 0.04819637317793646 actual  0.09763616107023253
predicted 0.9949608406685639 actual  0.9298934126838743
predicted 0.33556844269441866 actual 0.34014
predicted -1.3054234651952525 actual -1.3012
predicted 0.9995059433603526 actual  0.9629550321199144
predicted 0.9852190831790003 actual  0.9264994622708123
predicted -1.3187129863430558 actual  -1.3321
predicted 0.04919807214262918 actual  0.09953950144951805
predicted 0.9949603225712215 actual  0.9285601338447277
predicted 0.33543445191721905 actual 0.34014
predicted -1.3054248930024022 actual -1.3012
predicted 0.9994956505136933 actual  0.9620985010706639
predicted 0.9850212265317267 actual  0.9252557983498445
predicted -1.3189108429903293 actual  -1.3328
predicted 0.05019880047490677

predicted 0.3326206455960272 actual 0.34014
predicted -1.3054548769525447 actual -1.3012
predicted 0.9992795007338475 actual  0.9466809421841542
predicted 0.9809075173468691 actual  0.9001337871462981
predicted -1.3230245521751869 actual  -1.3495
predicted 0.07099190660221595 actual  0.1392967529830316
predicted 0.9949489440105389 actual  0.8994070965418102
predicted 0.3324866548188276 actual 0.34014
predicted -1.3054563047596943 actual -1.3012
predicted 0.9992692078871882 actual  0.9458244111349037
predicted 0.9807135750155627 actual  0.8990144896174271
predicted -1.3232184945064933 actual  -1.3504
predicted 0.07197156978091734 actual  0.14100918080147692
predicted 0.994948427689596 actual  0.8980937173867553
predicted 0.332352664041628 actual 0.34014
predicted -1.305457732566844 actual -1.3012
predicted 0.9992589150405289 actual  0.9451820128479658
predicted 0.980519808282417 actual  0.8978951920885564
predicted -1.3234122612396388 actual  -1.3512
predicted 0.0729502884249601 actual 

predicted 0.3295388577204362 actual 0.34014
predicted -1.3054877165169865 actual -1.3012
predicted 0.9990427652606831 actual  0.9297644539614561
predicted 0.9764909186516985 actual  0.8760999818735983
predicted -1.3274411508703572 actual  -1.3714
predicted 0.09328715253204366 actual  0.1763337897860283
predicted 0.994937087976558 actual  0.8696172693455495
predicted 0.32940486694323656 actual 0.34014
predicted -1.3054891443241363 actual -1.3012
predicted 0.9990324724140238 actual  0.9291220556745182
predicted 0.9763009650208837 actual  0.8751361423348485
predicted -1.327631104501172 actual  -1.3724
predicted 0.09424537061355749 actual  0.17782637889805158
predicted 0.9949365734107956 actual  0.8683436895588632
predicted 0.32927087616603695 actual 0.34014
predicted -1.305490572131286 actual -1.3012
predicted 0.9990221795673645 actual  0.9282655246252677
predicted 0.9761111824636742 actual  0.8741723027960986
predicted -1.3278208870583814 actual  -1.3735
predicted 0.09520266939510812 act

...

predicted -0.04670724465602139 actual 0.34014
predicted -1.3094969989931926 actual -1.3012
predicted 0.9701404518413096 actual  0.029122055674518203
predicted 0.7627527507917053 actual  0.0018978311433966275
predicted -1.541179318730353 actual  -1.5708
predicted 1.102223115740018 actual  0.9834889561018818
predicted 0.9936733821445888 actual  3.66817842164747e-05
predicted -0.04684123543322098 actual 0.34014
predicted -1.3094984268003422 actual -1.3012
predicted 0.9701301589946503 actual  0.029122055674518203
predicted 0.7627280123730388 actual  0.001878865268601871
predicted -1.5412040571490195 actual  -1.5708
predicted 1.1023151039460615 actual  0.9834889561018818
predicted 0.9936729681640849 actual  3.630767027075236e-05
predicted -0.04697522621042057 actual 0.34014
predicted -1.3094998546074919 actual -1.3012
predicted 0.970119866147991 actual  0.028907922912205567
predicted 0.7627032887980457 actual  0.0018602103097873554
predicted -1.5412287807240126 actual  -1.5708
predicted 1.1

predicted -1.3095298385576344 actual -1.3012
predicted 0.9699037163681452 actual  0.028693790149892935
predicted 0.762187503241794 actual  0.00146565793086037
predicted -1.5417445662802645 actual  -1.5708
predicted 1.1043187475888214 actual  0.9834889561018818
predicted 0.9936638638988994 actual  2.8212478640960373e-05
predicted -0.04992302330881155 actual 0.34014
predicted -1.309531266364784 actual -1.3012
predicted 0.9698934235214859 actual  0.028693790149892935
predicted 0.762163103464858 actual  0.0014470029720458546
predicted -1.5417689660572005 actual  -1.5708
predicted 1.1044089136200834 actual  0.9834889561018818
predicted 0.9936634502182378 actual  2.7848314520418826e-05
predicted -0.05005701408601114 actual 0.34014
predicted -1.3095326941719336 actual -1.3012
predicted 0.9698831306748266 actual  0.028693790149892935
predicted 0.7621387182812301 actual  0.001428037097251098
predicted -1.5417933512408284 actual  -1.5708
predicted 1.1044990010522444 actual  0.9834889561018818
pr

predicted -0.05287082040720253 actual 0.34014
predicted -1.3095626781220764 actual -1.3012
predicted 0.9696669808949808 actual  0.028265524625267664
predicted 0.7616299814592943 actual  0.0010337956343043565
predicted -1.5423020880627645 actual  -1.5708
predicted 1.1063727720818664 actual  0.9834889561018818
predicted 0.9936543525059146 actual  1.9955104693924314e-05
predicted -0.05300481118440212 actual 0.34014
predicted -1.309564105929226 actual -1.3012
predicted 0.9696566880483215 actual  0.028265524625267664
predicted 0.7616059146156043 actual  0.0010148297595096
predicted -1.5423261549064544 actual  -1.5708
predicted 1.1064611433615874 actual  0.9834889561018818
predicted 0.9936539391211366 actual  1.9600890398563554e-05
predicted -0.05313880196160171 actual 0.34014
predicted -1.3095655337363756 actual -1.3012
predicted 0.9696463952016622 actual  0.028265524625267664
predicted 0.7615818621193792 actual  0.0009961748006950845
predicted -1.5423502074026796 actual  -1.5708
predicted 

predicted 0.9694405382684758 actual  0.028051391862955032
predicted 0.761103808918457 actual  0.0006205882965628557
predicted -1.5428282606036026 actual  -1.5708
predicted 1.1082991344547635 actual  0.9834889561018818
predicted 0.9936452609806862 actual  1.2255916921780796e-05
predicted -0.055952608282793095 actual 0.34014
predicted -1.3095955176865182 actual -1.3012
predicted 0.9694302454218164 actual  0.028051391862955032
predicted 0.761080055286069 actual  0.0006019333377483403
predicted -1.5428520142359905 actual  -1.5708
predicted 1.1083858141952645 actual  0.9834889561018818
predicted 0.9936448478752812 actual  1.1910458368794189e-05
predicted -0.056086599059992685 actual 0.34014
predicted -1.3095969454936678 actual -1.3012
predicted 0.9694199525751571 actual  0.028051391862955032
predicted 0.7610563157701383 actual  0.0005829674629535837
predicted -1.542875753751921 actual  -1.5708
predicted 1.108472417625163 actual  0.9834889561018818
predicted 0.9936444347824912 actual  1.1565

predicted -0.05890040538118407 actual 0.34014
predicted -1.3096269294438103 actual -1.3012
predicted 0.9692038027953114 actual  0.027623126338329765
predicted 0.7605610284818617 actual  0.0001887260000068394
predicted -1.5433710410401975 actual  -1.5708
predicted 1.1102735498886698 actual  0.9834889561018818
predicted 0.9936357627351705 actual  4.414446482754783e-06
predicted -0.05903439615838366 actual 0.34014
predicted -1.30962835725096 actual -1.3012
predicted 0.969193509948652 actual  0.027623126338329765
predicted 0.7605375969099193 actual  0.00017007104119232678
predicted -1.54339447261214 actual  -1.5708
predicted 1.1103584870337455 actual  0.9834889561018818
predicted 0.9936353499180854 actual  4.078340806210772e-06
predicted -0.05916838693558325 actual 0.34014
predicted -1.3096297850581096 actual -1.3012
predicted 0.9691832171019927 actual  0.027623126338329765
predicted 0.7605141792171756 actual  0.0001511051663975702
predicted -1.5434178903048836 actual  -1.5708
predicted 1.

## ω traj


In [ ]:
ω_dnn = np.append(ω_dnn,omega1_an)
n = np.append(ω_dnn,omega1_an)
c2 = (t_ref*m0*g0*l_com/I_z)
kω1 = - T1*np.sin(b1_an)*c2 # these values obtained are w.r.t tprime so to get w.r.t we need to divide by 32
kω2 = - T1_int*np.sin(b1_int_an)*c2
kω3 = - T1_int*np.sin(b1_int_an)*c2
kω4 = - T2*np.sin(b2_an)*c2
omega2_an = omega1_an + h_step/6 * (kω1 + 2*kω2 + 2*kω3 + kω4) 
omega_int = (omega1_an + omega2_an)/2
ω_int_dnn = np.append(ω_int_dnn,omega_int)
print("predicted",omega2_an,"actual", AntiNorm(ip_traj[1,3],omega))

In [ ]:
b_int_dnn = np.append(b_int_dnn,b1_int_an)
T_int_dnn = np.append(T_int_dnn,T1_int)

## θ traj


In [ ]:
theta1_an = AntiNorm(theta1,theta)
θ_dnn = np.append(θ_dnn,theta1_an)
kθ1 = t_ref*omega1_an
kθ2 = t_ref*omega_int # intermediate trajectories
kθ3 = t_ref*omega_int
kθ4 = t_ref*omega2_an
theta2_an =  theta1_an + h_step/6* (kθ1 +2*kθ2 + 2*kθ3 + kθ4)
theta_int = (theta1_an + theta2_an)/2
θ_int_dnn = np.append(θ_int_dnn,theta_int)
print("predicted",theta2_an,"actual", AntiNorm(ip_traj[1,-1],theta))


## m traj 


In [ ]:
m_dnn = np.append(m_dnn,m1)
km1 = -(t_ref/Isp)*T1
km2 = -(t_ref/Isp)*T1_int
km3 = -(t_ref/Isp)*T1_int
km4 = -(t_ref/Isp)*T2
m2 = m1 + h_step/6* (km1 + 2*km2 + 2*km3 + km4)
m1_int = (m2 + m1)/2
m_int_dnn = np.append(m_int_dnn,m1_int)
print("predicted", m2 ,"actual ", m_norm[idx_i+1])

## v traj


In [ ]:
v_dnn = np.append(v_dnn,v1)

c1 = 0.5*1.225*Cd*(v0)**2 *S_ref / (m0*g0)
D1 = c1 * v1**2
# Intermediate of r1 is obtained from Euler method

kv1 = -(g0*t_ref/v0) *(((T1 * np.cos(b1_an - gamma1_an + theta1_an) + D1)/m1) + (np.sin(gamma1_an)/(r1**2)))

D1_int = c1 * (v1 + h_step*kv1/2)**2
kv2 = -(g0*t_ref/v0) *(((T1_int * np.cos(b1_int_an - g1_int_an + t1_int_an) + D1_int/m1_int) + (np.sin(g1_int_an)/(r1_int**2))))

D2_int = c1 * (v1 + h_step*kv2/2)**2
kv3 = -(g0*t_ref/v0) *(((T1_int * np.cos(b1_int_an - g1_int_an + t1_int_an) + D2_int/m1_int) + (np.sin(g1_int_an)/(r1_int**2))))

D3_int = c1 * (v1+h_step*kv3)**2
gamma2_an = gamma1_an + -(g0*t_ref/v0) *(((T1 * np.sin(b1_an - gamma1_an + theta1_an))/(m1*v1)) + (np.cos(gamma1_an)/(r1**2 * v1)))
kv4 = -(g0*t_ref/v0) *(((T2 * np.cos(b2_an - gamma2_an + theta2_an) + D3_int)/m2) + (np.sin(gamma2_an)/(r1**2)))

v2 = v1 + h_step/6 * ( kv1 + 2*kv2 + 2*kv3 + kv4)

v1_int = (v1 + v2)/2
v_int_dnn = np.append(v_int_dnn,v1_int)
print("predicted", v2 ,"actual ", v_norm[idx_i+1])

In [ ]:
# kθ2 = t_ref*AntiNorm(ip_int.reshape(1,7)[0,3],omega) # intermediate trajectories
# kθ3 = t_ref*AntiNorm(ip_int.reshape(1,7)[0,3],omega)
# kθ4 = t_ref*ip_traj[1,3]


## γ traj


In [ ]:
γ_dnn = np.append(γ_dnn,gamma1_an)

g1 = gamma1_an
kγ1 = -(g0*t_ref/v0) *(((T1 * np.sin(b1_an - g1 + theta1_an))/(m1*v1)) + (np.cos(g1)/(r1**2 * v1)))

g2 = g1_int_an + (h_step * kγ1 / 2)
kγ2 = -(g0*t_ref/v0) *((T1_int * np.sin(b1_int_an - g2 + t1_int_an)/(m1_int*v1_int)) + (np.cos(g2)/(r1_int**2*v1_int)))

g3 = g1_int_an + (h_step * kγ2 / 2)
kγ3 = -(g0*t_ref/v0) *((T1_int * np.sin(b1_int_an - g3 + t1_int_an)/(m1_int*v1_int)) + (np.cos(g3)/(r1_int**2*v1_int)))

g4 = g1_int_an + (h_step * kγ3)
kγ4 = -(g0*t_ref/v0) *(((T2 * np.sin(b2_an - g4 + theta2_an) + D3_int)/(m2*v2)) + (np.cos(g4)/(r1**2*v2)))

gamma2_an = gamma1_an + h_step/6 * ( kv1 + 2*kv2 + 2*kv3 + kv4)
g1_int_an = (gamma2_an + gamma1_an)/2
γ_int_dnn = np.append(γ_int_dnn,g1_int_an) 
print("predicted", gamma2_an ,"actual ", AntiNorm(gamma_norm[idx_i+1],gamma))

## S traj 


In [ ]:
s_dnn = np.append(s_dnn,si)
c4 = (v0*t_ref/s0)
ks1 = c4 * v1 * np.cos(gamma1_an)
ks2 = c4 * v1_int * np.cos(g1_int_an)
ks3 = ks2
ks4 = c4 * v2 * np.cos(gamma2_an)
s2 = si + (h_step/6) * (ks1 + 2*ks2 + 2*ks3 + ks4)
s1_int = (s1+s2)/2
s_int_dnn = np.append(s_int_dnn,s1_int)
print("predicted", s2 ,"actual ", s_norm[idx_i+1])

## r traj

In [ ]:
r_dnn = np.append(r_dnn,r1)
c3 = (v0*t_ref/R0)
kr1 = c3 * v1 * np.sin(gamma1_an)
kr2 = c3 * v1_int * np.sin(g1_int_an)
kr3 = kr2
kr4 = c3 * v2 * np.sin(gamma2_an)

r2 = r1 + (h_step/6) * (kr1 + 2*kr2 + 2*kr3 + kr4)
r1_int = (r1 + r2)/2
r_int_dnn = np.append(r_int_dnn,r1_int)
print("predicted", r2 , "actual ", r_norm[idx_i+1])

## h traj

In [ ]:
h_dnn = np.append(h_dnn,h1)
h2 = norm(AntiNorm(r2,r)-6378000,h)
h1_int = (h1+h2)/2
h_int_dnn = np.append(h_int_dnn,h1_int)

## changing values to next iterate to run for loop


In [ ]:
r1 = r2;
si = s2;
gamma1_an = gamma2_an;
h1 = h2;
m1 = m2;
theta1_an = theta2_an;
omega1_an = omega2_an;
v1= v2;

# Controls intermediate traj

In [ ]:
ip = np.array([h1,v1,s1,omega1,gamma1,m1,theta1]).reshape(1,7)


In [ ]:
##############################################################################
si = s_norm[idx_i]
gamma1_int = ip_mid[0,4]
theta1_int = ip_mid[0,6]
# taken values from intermediate stuff to send as inputs, but these should be same as initial values. As time progresses 
# they get updated by iterations.
g1_int_an = AntiNorm(gamma1_int,gamma) # gamma _intermediate antinormalized
t1_int_an = AntiNorm(theta1_int,theta) # theta _intermediate antinormalized
r1_int = r1 + h_step/2 * v1* np.sin(g1_int_an) * (t_ref *v0/R0)
##############################################################################

# def RK4():
for _ in range(3003):
    ω_dnn = np.append(ω_dnn,omega1_an)
    n = np.append(ω_dnn,omega1_an)
    c2 = (t_ref*m0*g0*l_com/I_z)
    kω1 = - T1*np.sin(b1_an)*c2 # these values obtained are w.r.t tprime so to get w.r.t we need to divide by 32
    kω2 = - T1_int*np.sin(b1_int_an)*c2
    kω3 = - T1_int*np.sin(b1_int_an)*c2
    kω4 = - T2*np.sin(b2_an)*c2
    omega2_an = omega1_an + h_step/6 * (kω1 + 2*kω2 + 2*kω3 + kω4) 
    omega_int = (omega1_an + omega2_an)/2
    ω_int_dnn = np.append(ω_int_dnn,omega_int)
    print("predicted",omega2_an,"actual", AntiNorm(ip_traj[1,3],omega))

    b_int_dnn = np.append(b_int_dnn,b1_int_an)
    T_int_dnn = np.append(T_int_dnn,T1_int)

    theta1_an = AntiNorm(theta1,theta)
    θ_dnn = np.append(θ_dnn,theta1_an)
    kθ1 = t_ref*omega1_an
    kθ2 = t_ref*omega_int # intermediate trajectories
    kθ3 = t_ref*omega_int
    kθ4 = t_ref*omega2_an
    theta2_an =  theta1_an + h_step/6* (kθ1 +2*kθ2 + 2*kθ3 + kθ4)
    theta_int = (theta1_an + theta2_an)/2
    θ_int_dnn = np.append(θ_int_dnn,theta_int)
    print("predicted",theta2_an,"actual", AntiNorm(ip_traj[1,-1],theta))

    m_dnn = np.append(m_dnn,m1)
    km1 = -(t_ref/Isp)*T1
    km2 = -(t_ref/Isp)*T1_int
    km3 = -(t_ref/Isp)*T1_int
    km4 = -(t_ref/Isp)*T2
    m2 = m1 + h_step/6* (km1 + 2*km2 + 2*km3 + km4)
    m1_int = (m2 + m1)/2
    m_int_dnn = np.append(m_int_dnn,m1_int)
    print("predicted", m2 ,"actual ", m_norm[idx_i+1])

    v_dnn = np.append(v_dnn,v1)

    c1 = 0.5*1.225*Cd*(v0)**2 *S_ref / (m0*g0)
    D1 = c1 * v1**2
    # Intermediate of r1 is obtained from Euler method

    kv1 = -(g0*t_ref/v0) *(((T1 * np.cos(b1_an - gamma1_an + theta1_an) + D1)/m1) + (np.sin(gamma1_an)/(r1**2)))

    D1_int = c1 * (v1 + h_step*kv1/2)**2
    kv2 = -(g0*t_ref/v0) *(((T1_int * np.cos(b1_int_an - g1_int_an + t1_int_an) + D1_int/m1_int) + (np.sin(g1_int_an)/(r1_int**2))))

    D2_int = c1 * (v1 + h_step*kv2/2)**2
    kv3 = -(g0*t_ref/v0) *(((T1_int * np.cos(b1_int_an - g1_int_an + t1_int_an) + D2_int/m1_int) + (np.sin(g1_int_an)/(r1_int**2))))

    D3_int = c1 * (v1+h_step*kv3)**2
    gamma2_an = gamma1_an + -(g0*t_ref/v0) *(((T1 * np.sin(b1_an - gamma1_an + theta1_an))/(m1*v1)) + (np.cos(gamma1_an)/(r1**2 * v1)))
    kv4 = -(g0*t_ref/v0) *(((T2 * np.cos(b2_an - gamma2_an + theta2_an) + D3_int)/m2) + (np.sin(gamma2_an)/(r1**2)))

    v2 = v1 + h_step/6 * ( kv1 + 2*kv2 + 2*kv3 + kv4)

    v1_int = (v1 + v2)/2
    v_int_dnn = np.append(v_int_dnn,v1_int)
    print("predicted", v2 ,"actual ", v_norm[idx_i+1])

    γ_dnn = np.append(γ_dnn,gamma1_an)

    g1 = gamma1_an
    kγ1 = -(g0*t_ref/v0) *(((T1 * np.sin(b1_an - g1 + theta1_an))/(m1*v1)) + (np.cos(g1)/(r1**2 * v1)))

    g2 = g1_int_an + (h_step * kγ1 / 2)
    kγ2 = -(g0*t_ref/v0) *((T1_int * np.sin(b1_int_an - g2 + t1_int_an)/(m1_int*v1_int)) + (np.cos(g2)/(r1_int**2*v1_int)))

    g3 = g1_int_an + (h_step * kγ2 / 2)
    kγ3 = -(g0*t_ref/v0) *((T1_int * np.sin(b1_int_an - g3 + t1_int_an)/(m1_int*v1_int)) + (np.cos(g3)/(r1_int**2*v1_int)))

    g4 = g1_int_an + (h_step * kγ3)
    kγ4 = -(g0*t_ref/v0) *(((T2 * np.sin(b2_an - g4 + theta2_an) + D3_int)/(m2*v2)) + (np.cos(g4)/(r1**2*v2)))

    gamma2_an = gamma1_an + h_step/6 * ( kv1 + 2*kv2 + 2*kv3 + kv4)
    g1_int_an = (gamma2_an + gamma1_an)/2
    γ_int_dnn = np.append(γ_int_dnn,g1_int_an) 
    print("predicted", gamma2_an ,"actual ", AntiNorm(gamma_norm[idx_i+1],gamma))

    s_dnn = np.append(s_dnn,si)
    c4 = (v0*t_ref/s0)
    ks1 = c4 * v1 * np.cos(gamma1_an)
    ks2 = c4 * v1_int * np.cos(g1_int_an)
    ks3 = ks2
    ks4 = c4 * v2 * np.cos(gamma2_an)
    s2 = si + (h_step/6) * (ks1 + 2*ks2 + 2*ks3 + ks4)
    s1_int = (s1+s2)/2
    s_int_dnn = np.append(s_int_dnn,s1_int)
    print("predicted", s2 ,"actual ", s_norm[idx_i+1])

    r_dnn = np.append(r_dnn,r1)
    c3 = (v0*t_ref/R0)
    kr1 = c3 * v1 * np.sin(gamma1_an)
    kr2 = c3 * v1_int * np.sin(g1_int_an)
    kr3 = kr2
    kr4 = c3 * v2 * np.sin(gamma2_an)

    r2 = r1 + (h_step/6) * (kr1 + 2*kr2 + 2*kr3 + kr4)
    r1_int = (r1 + r2)/2
    r_int_dnn = np.append(r_int_dnn,r1_int)
    print("predicted", r2 , "actual ", r_norm[idx_i+1])

    h_dnn = np.append(h_dnn,h1)
    h2 = norm(AntiNorm(r2,r)-6378000,h)
    h1_int = (h1+h2)/2
    h_int_dnn = np.append(h_int_dnn,h1_int)

    r1 = r2;
    si = s2;
    gamma1_an = gamma2_an;
    h1 = h2;
    m1 = m2;
    theta1_an = theta2_an;
    omega1_an = omega2_an;
    v1= v2;

    # ip = np.array([h1,v1,s1,omega1,gamma1,m1,theta1]).reshape(1,7)


## ω traj
ω_dnn = np.append(ω_dnn,omega1_an)
c2 = (t_ref*m0*g0*l_com/I_z)
kω1 = - T1*np.sin(b1_an)*c2 # these values obtained are w.r.t tprime so to get w.r.t we need to divide by 32
kω2 = - T1_int*np.sin(b1_int_an)*c2
kω3 = - T1_int*np.sin(b1_int_an)*c2
kω4 = - T2*np.sin(b2_an)*c2
omega2_an = omega1_an + h_step/6 * (kω1 + 2*kω2 + 2*kω3 + kω4) 
print("predicted",omega2_an,"actual", AntiNorm(ip_traj[1,3],omega))

omega_int = (omega1_an + omega2_an)/2
ω_int_dnn = np.append(ω_int_dnn,omega_int)

## θ traj

theta1_an = AntiNorm(theta1,theta)
θ_dnn = np.append(θ_dnn,theta1_an)
kθ1 = t_ref*omega1_an
kθ2 = t_ref*omega_int # intermediate trajectories
kθ3 = t_ref*omega_int
kθ4 = t_ref*omega2_an
theta2_an =  theta1_an + h_step/6* (kθ1 +2*kθ2 + 2*kθ3 + kθ4)
theta_int = (theta1_an + theta2_an)/2
print("predicted",theta2_an,"actual", AntiNorm(ip_traj[1,-1],theta))

## m traj 
m_dnn = np.append(m_nn,m1)
km1 = -(t_ref/Isp)*T1
km2 = -(t_ref/Isp)*T1_int
km3 = -(t_ref/Isp)*T1_int
km4 = -(t_ref/Isp)*T2
m2 = m1 + h_step/6* (km1 + 2*km2 + 2*km3 + km4)
print("predicted", m2 ,"actual ", m_norm[idx_i+1])

## v traj
v_dnn = np.append(v_dnn,v1)
gamma1_int = ip_mid[0,4]
theta1_int = ip_mid[0,3]
m1_int = (m1+m2)/2;
g1_int_an = AntiNorm(gamma1_int,gamma) # gamma _intermediate antinormalized
t1_int_an = AntiNorm(theta1_int,theta) # theta _intermediate antinormalized
c1 = 0.5*1.225*Cd*(v0)**2 *S_ref / (m0*g0)
D1 = c1 * v1**2
# Intermediate of r1 is obtained from Euler method
r1_int = r1 + h_step/2 * v1* np.sin(g1_int_an) * (t_ref *v0/R0)

kv1 = -(g0*t_ref/v0) *(((T1 * np.cos(b1_an - g1_int_an + t1_int_an) + D1)/m1) + (np.sin(gamma1_an)/(r1**2)))

D1_int = c1 * (v1 + h_step*kv1/2)**2
kv2 = -(g0*t_ref/v0) *(((T1_int * np.cos(b1_int_an - g1_int_an + t1_int_an) + D1_int/m1_int) + (np.sin(g1_int_an)/(r1_int**2))))

D2_int = c1 * (v1 + h_step*kv2/2)**2
kv3 = -(g0*t_ref/v0) *(((T1_int * np.cos(b1_int_an - g1_int_an + t1_int_an) + D2_int/m1_int) + (np.sin(g1_int_an)/(r1_int**2))))

D3_int = c1 * (v1+h_step*kv3)**2
# gamma2_an = gamma1_an + 
kv4 = -(g0*t_ref/v0) *(((T2 * np.cos(b2_an - gamma2_an + theta2_an) + D3_int)/m2) + (np.sin(gamma2_an)/(r1**2)))

v2 = v1 + h_step/6 * ( kv1 + 2*kv2 + 2*kv3 + kv4)

## γ traj
γ_dnn = np.append(γ_dnn,gamma1_an)
gamma1_int = ip_mid[0,4]
theta1_int = ip_mid[0,3]
m1_int = (m1+m2)/2;
g1_int_an = AntiNorm(gamma1_int,gamma) # gamma _intermediate antinormalized
t1_int_an = AntiNorm(theta1_int,theta) # theta _intermediate antinormalized
# Intermediate of r1 is obtained from Euler method
r1_int = r1 + h_step/2 * v1* np.sin(g1_int_an) * (t_ref *v0/R0)
v1_int = (v1+v2)/2
g1 = g1_int_an
kγ1 = -(g0*t_ref/v0) *(((T1 * np.sin(b1_an - g1 + t1_int_an))/(m1*v1)) + (np.cos(gamma1_an)/(r1**2 * v1)))

g2 = g1 + (h_step * kγ1 / 2)
kγ2 = -(g0*t_ref/v0) *((T1_int * np.sin(b1_int_an - g2 + t1_int_an)/(m1_int*v1_int)) + (np.cos(g2)/(r1_int**2*v1_int)))

g3 = g1_int_an + (h_step * kγ2 / 2)
kγ3 = -(g0*t_ref/v0) *((T1_int * np.sin(b1_int_an - g3 + t1_int_an)/(m1_int*v1_int)) + (np.cos(g3)/(r1_int**2*v1_int)))

g4 = g1_int_an + (h_step * kγ3)
kγ4 = -(g0*t_ref/v0) *(((T2 * np.sin(b2_an - g4 + theta2_an) + D3_int)/(m2*v2)) + (np.cos(g4)/(r1**2*v2)))

gamma2_an = gamma1_an + h_step/6 * ( kv1 + 2*kv2 + 2*kv3 + kv4)

## S traj 
s_dnn = np.append(s_dnn,s1)
c4 = (v0*t_ref/s0)
gamma1_int_an = AntiNorm(gamma1_int,gamma)

ks1 = c4 * v1 * np.cos(gamma1_an)
ks2 = c4 * v1_int * np.cos(gamma1_int_an)
ks3 = ks2
ks4 = c4 * v2 * np.cos(gamma2_an)
si = s_norm[idx_i]
s2 = si + (h_step/6) * (ks1 + 2*ks2 + 2*ks3 + ks4)

## r traj 
r_dnn = np.append(r_dnn,r1)
c3 = (v0*t_ref/R0)
gamma1_int_an = AntiNorm(gamma1_int,gamma)

kr1 = c3 * v1 * np.sin(gamma1_an)
kr2 = c3 * v1_int * np.sin(gamma1_int_an)
kr3 = kr2
kr4 = c3 * v2 * np.sin(gamma2_an)

r2 = r1 + (h_step/6) * (kr1 + 2*kr2 + 2*kr3 + kr4)
print("predicted", r2 , "actual ", r_norm[idx_i+1])

## h traj 
h_dnn = np.append(h_dnn,h1)
h2 = (r2*R0-R0)

## changing values to next iterate to run for loop
r1 = r2;
s1 = s2;
gamma1_an = gamma2_an;
h1 = h2;
m1 = m2;
theta1_an = theta2_an;
omega1_an = omega2_an;
v1= v2;


In [ ]:
df.head(1)